In [21]:
from sklearn.ensemble import RandomForestRegressor
from pandas import Timestamp
from functools import partial
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from src.helpers import get_data, clean_data, split_data, train_model, save_model, predict
%matplotlib inline

In [22]:
train_set = get_data('../data/raw/train_set.csv')
train_set.head()

,created_at,site,pm2_5,pm10,s2_pm2_5,s2_pm10,humidity,temp,lat,long,altitude,greenness,landform_90m,landform_270m,population,dist_major_road,ref_pm2_5
0,2019-12-02 08:00:00+03:00,USEmbassy,74.088667,83.767111,0.000000,0.000000,0.962917,21.475000,0.299255,32.592686,1199,4374,21,14,6834,130,95.0
1,2020-03-17 02:00:00+03:00,USEmbassy,88.350233,96.003023,86.993256,107.929767,0.967667,19.233333,0.299255,32.592686,1199,4374,21,14,6834,130,139.0
2,2019-10-13 05:00:00+03:00,USEmbassy,47.540513,58.672308,48.684103,60.505128,0.999250,18.916667,0.299255,32.592686,1199,4374,21,14,6834,130,46.0
3,2020-04-04 11:00:00+03:00,USEmbassy,24.567619,27.151429,23.406667,27.316429,0.783500,25.475000,0.299255,32.592686,1199,4374,21,14,6834,130,33.0
4,2020-03-22 20:00:00+03:00,USEmbassy,46.644857,57.555143,46.195143,60.075714,0.721833,22.916667,0.299255,32.592686,1199,4374,21,14,6834,130,60.0


In [23]:
train_set = clean_data(train_set)
train_set.head()

,pm2_5,pm10,s2_pm2_5,s2_pm10,humidity,temp,lat,long,altitude,greenness,landform_270m,population,ref_pm2_5,hour_of_day
0,74.088667,83.767111,74.088667,83.767111,0.962917,21.475000,0.299255,32.592686,1199,4374,14,6834,95.0,8
1,88.350233,96.003023,86.993256,107.929767,0.967667,19.233333,0.299255,32.592686,1199,4374,14,6834,139.0,2
2,47.540513,58.672308,48.684103,60.505128,0.999250,18.916667,0.299255,32.592686,1199,4374,14,6834,46.0,5
3,24.567619,27.151429,23.406667,27.316429,0.783500,25.475000,0.299255,32.592686,1199,4374,14,6834,33.0,11
4,46.644857,57.555143,46.195143,60.075714,0.721833,22.916667,0.299255,32.592686,1199,4374,14,6834,60.0,20


In [24]:
X = train_set.drop(columns = ['ref_pm2_5'], axis=1)
y = train_set['ref_pm2_5']

In [25]:
Xtrain, Xtest, ytrain, ytest = split_data(X, y)

In [26]:
rfr = RandomForestRegressor()

In [27]:
train_model(rfr, Xtrain, ytrain)

In [28]:
save_model(rfr, '../models/rfr.joblib')

In [29]:
rmse, ypred = predict(rfr, Xtest, ytest)
rmse

13.38

In [30]:
rfr2 = RandomForestRegressor( #From GridSearchCV
    max_depth=20,
    max_features='auto',
    min_samples_leaf=2,
    min_samples_split=2,
    n_estimators=200,
    oob_score=False)

In [31]:
train_model(rfr2, Xtrain, ytrain)

In [32]:
save_model(rfr2, '../models/rfr_grid_search.joblib')

In [33]:
rmse, ypred2 = predict(rfr2, Xtest, ytest)
rmse

13.16

#### Testing on held-out data

In [34]:
Xactual = get_data('../data/raw/test_set.csv')
Xactual.head()

,created_at,site,pm2_5,pm10,s2_pm2_5,s2_pm10,humidity,temp,lat,long,altitude,greenness,landform_90m,landform_270m,population,dist_major_road
0,2020-03-20 12:00:00+03:00,USEmbassy,9.903636,10.259545,8.852727,10.097727,0.748000,26.491667,0.299255,32.592686,1199,4374,21,14,6834,130
1,2019-10-12 06:00:00+03:00,USEmbassy,38.428333,45.940714,40.087143,48.159762,0.995000,18.225000,0.299255,32.592686,1199,4374,21,14,6834,130
2,2019-12-13 17:00:00+03:00,USEmbassy,13.278000,14.967333,0.000000,0.000000,0.687500,24.891667,0.299255,32.592686,1199,4374,21,14,6834,130
3,2019-11-13 09:00:00+03:00,USEmbassy,36.304000,40.366750,89.182500,93.572500,0.868417,22.616667,0.299255,32.592686,1199,4374,21,14,6834,130
4,2019-08-07 19:00:00+03:00,USEmbassy,56.686410,67.412308,55.048974,66.089487,0.782083,23.350000,0.299255,32.592686,1199,4374,21,14,6834,130


In [35]:
Xactual = clean_data(Xactual)
Xactual.head()

,pm2_5,pm10,s2_pm2_5,s2_pm10,humidity,temp,lat,long,altitude,greenness,landform_270m,population,hour_of_day
0,9.903636,10.259545,8.852727,10.097727,0.748000,26.491667,0.299255,32.592686,1199,4374,14,6834,12
1,38.428333,45.940714,40.087143,48.159762,0.995000,18.225000,0.299255,32.592686,1199,4374,14,6834,6
2,13.278000,14.967333,13.278000,14.967333,0.687500,24.891667,0.299255,32.592686,1199,4374,14,6834,17
3,36.304000,40.366750,89.182500,93.572500,0.868417,22.616667,0.299255,32.592686,1199,4374,14,6834,9
4,56.686410,67.412308,55.048974,66.089487,0.782083,23.350000,0.299255,32.592686,1199,4374,14,6834,19


In [36]:
yactual = get_data('../data/raw/test_solution.csv')
yactual.head()

,ref_pm2_5
0,30.0
1,44.0
2,27.0
3,48.0
4,57.0


In [37]:
rmse_heldout1, ypred_heldout1 = predict(rfr, Xactual, yactual)
rmse_heldout1

13.35

In [38]:
rmse_heldout2, ypred_heldout2 = predict(rfr2, Xactual, yactual)
rmse_heldout2

13.12

Text(0, 0.5, 'Predictions ')

In [39]:
g=plt.scatter(yactual, ypred_heldout2)
g.axes.set_xlabel('Actual Values ')
g.axes.set_ylabel('Predictions ')

In [42]:
ytest.shape, ypred.shape, ypred2.shape

((4920,), (4920,), (4920,))

In [43]:
y_df = Xtest[['pm2_5', 'pm10', 'altitude', 'hour_of_day']]
y_df.head()

,pm2_5,pm10,altitude,hour_of_day
533,2.587381,2.745000,1199,4
3883,21.706000,23.131333,1199,11
932,30.479333,34.863333,1233,19
6946,197.810682,218.537273,1191,7
4439,23.544667,25.238222,1199,16


In [44]:
y_df['ref_pm2_5'] = ytest
y_df.head()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,pm2_5,pm10,altitude,hour_of_day,ref_pm2_5
533,2.587381,2.745000,1199,4,19.0
3883,21.706000,23.131333,1199,11,35.0
932,30.479333,34.863333,1233,19,23.7
6946,197.810682,218.537273,1191,7,179.3
4439,23.544667,25.238222,1199,16,44.0


In [45]:
y_df['ypred'] = ypred
y_df.head()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,pm2_5,pm10,altitude,hour_of_day,ref_pm2_5,ypred
533,2.587381,2.745000,1199,4,19.0,18.620
3883,21.706000,23.131333,1199,11,35.0,36.990
932,30.479333,34.863333,1233,19,23.7,23.769
6946,197.810682,218.537273,1191,7,179.3,183.775
4439,23.544667,25.238222,1199,16,44.0,33.433


In [46]:
y_df['diff_ref'] = abs(y_df['ref_pm2_5'] - y_df['ypred'])
y_df['diff2_lc'] = abs(y_df['ref_pm2_5'] - y_df['pm2_5'])
y_df.head()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,pm2_5,pm10,altitude,hour_of_day,ref_pm2_5,ypred,diff_ref,diff2_lc
533,2.587381,2.745000,1199,4,19.0,18.620,0.380,16.412619
3883,21.706000,23.131333,1199,11,35.0,36.990,1.990,13.294000
932,30.479333,34.863333,1233,19,23.7,23.769,0.069,6.779333
6946,197.810682,218.537273,1191,7,179.3,183.775,4.475,18.510682
4439,23.544667,25.238222,1199,16,44.0,33.433,10.567,20.455333


In [51]:
y_df.to_csv(r'../data/raw/actual_vs_pred.csv', index=False)

In [48]:
diff_df = y_df[(y_df['diff_ref']>=50)]
diff_df.shape

(64, 8)

In [49]:
diff_df

,pm2_5,pm10,altitude,hour_of_day,ref_pm2_5,ypred,diff_ref,diff2_lc
6987,42.506765,52.645000,1191,22,147.2,24.377,122.823,104.693235
6728,101.246364,110.517045,1191,2,0.1,102.802,102.702,101.146364
7300,40.124474,49.855000,1191,18,109.7,58.038,51.662,69.575526
5065,60.683023,71.993488,1191,2,2.6,53.481,50.881,58.083023
7336,113.693256,124.165116,1191,23,1.4,109.773,108.373,112.293256
...,...,...,...,...,...,...,...,...
815,127.180698,141.187209,1191,7,80.1,130.423,50.323,47.080698
5106,52.800909,90.825000,1199,11,197.0,135.550,61.450,144.199091
1867,103.015227,110.878182,1199,9,186.0,109.383,76.617,82.984773
734,57.267895,65.202105,1191,13,128.8,59.913,68.887,71.532105


In [50]:
diff_df['altitude'].value_counts()

1191    36
1199    21
1233     7
Name: altitude, dtype: int64